In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py

from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys
#sys.path.append("/Users/ajc/Core/PostProcessData/PyScript")
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata as rd
import os
#import cv2

In [2]:
#mesh_file='/Users/ajc/FUSE/ats-tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/test2'
path='/Users/ajc/FUSE1/simulations/tests/ats-tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/Srun1A'
path = '/Users/ajc/FUSE3/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic/test200/'
#path='/Users/ajc/FUSE1/simulations/tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/test0'

In [3]:
F= ['test1']
infile = [os.path.join(path,'visdump_surface_data.h5'), os.path.join(path,'visdump_surface_mesh.h5') ]

mesh = h5py.File(infile[1],'r')
mesh_data = list(mesh['0']['Mesh']['Nodes'])
vars = ['surface-total_component_concentration']#, 'surface-total_component_concentration']
Suffix = ['Tracer1']#, 'Tracer2']

In [4]:
Stream_C = dict()
for i, v in enumerate(vars):
    Stream_C[Suffix[i]] = rd.GetSurfVarFromVis_All(infile[0],vars[i],suffix='.cell.'+Suffix[i])[v]
cycles = len(Stream_C[Suffix[0]])
print (len(Stream_C[Suffix[0]]),Stream_C.keys())

46 dict_keys(['Tracer1'])


In [5]:
X_mesh = []
Y_mesh = []
Z_mesh = []

for m in mesh_data:#[:30]:
    X_mesh.append(m[0])
    Y_mesh.append(m[1])
    Z_mesh.append(m[2])
print (len(Stream_C[Suffix[0]]), len(Stream_C[Suffix[0]][0]), len(Z_mesh))

def subgrid_mesh(mesh_sg_data):
    X_sgmesh,Y_sgmesh,Z_sgmesh=[],[],[]
    for m in mesh_sg_data:
        X_sgmesh.append(m[0])
        Y_sgmesh.append(m[1])
        Z_sgmesh.append(m[2])
    return X_sgmesh,Y_sgmesh,Z_sgmesh

#def subgrid_C(sgfile,varname_sg):
#    sgC_data = rd.GetSurfVarFromVis_All(sgfile,varname_sg,suffix='.cell.'+Suffix[i])
#    sgConcentration = sgC_data[varname_sg]
#    #print ('subgrid C: ',sgConcentration)
#    return sgConcentration

def subgrid_C(sg_file,sg_var, suff):
    sgC_data = rd.GetSurfVarFromVis_All(sg_file,sg_var,suffix='.cell.'+suff)
    sgConcentration = sgC_data[sg_var]
    return sgConcentration

46 138 138


In [7]:
#path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/Coweeta/Srun1A/'
outfile='/Users/ajc/Core/PostProcessing/ReactiveTransport/Hyporheic/Coweeta/may26/test0A/figs2/'

In [8]:
#Write solute concentration images to a directory
#%matplotlib qt


"""
x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]
print (cycles)
count = 0
for k in range(0,cycles,1):
    
    fig = plt.figure(figsize=(8,4))#subplots(1,1,)
    ax = fig.gca(projection='3d',adjustable='box')
    
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*1

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    
    #ran = [f*1 for f in flux_interpolate] #Discharge_stream
    ran = Stream_C[Suffix[0]][k]
    ran = [r*1e-3 if r>0 else 1e-12 for r in ran]
    print (k, Suffix[0], len(ran))
    sc = ax.scatter(X_mesh,Y_mesh,z,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())
    
    
    plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
   
    #ax.set_zlim(0,0.0)
    #ax.set_zticks([])
    
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
    
    for x1 in range(0,len(x_orig),75):
        
        mesh_name = 'visdump_subgrid_%s_mesh.h5'%x1
        #print (mesh_name)
        
        mesh_sg = h5py.File(os.path.join(path,mesh_name),'r')
        mesh_sg_data = list(mesh_sg['0']['Mesh']['Nodes'])
        X_sgmesh,Y_sgmesh,Z_sgmesh = subgrid_mesh(mesh_sg_data)
        
        sg_infile = os.path.join(path,'visdump_subgrid_%s_data.h5'%x1)
        
        ran = subgrid_C(sg_infile,sg_var='subgrid_%s-total_component_concentration'%x1,suff=Suffix[0])[k][::-1]
        ran = [r*1e-3 if r>0 else 1e-12 for r in ran]
        Z_sgmesh = np.arange(0.05,1.05,0.05)#,dtype='int')
        Z_sgmesh = np.linspace(0,1.,20)#np.arange(0.905,1.05,0.005)#,dtype='int') # make is smaller compared to watershed
        #ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh, c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1)#,norm=matplotlib.colors.LogNorm())
        #ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1,norm=matplotlib.colors.LogNorm())
        #sc1 = ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=.1,norm=matplotlib.colors.LogNorm())
        #break
        
        cmap = cm.get_cmap('cool') # Get desired colormap - you can change this!
        cmap= cm.coolwarm
        max_height = 1#np.max(ran)   # get range of colorbars so we can normalize
        min_height = np.min(ran)
        # scale each z to [0,1], and get their rgb values
        rgba = np.array([cmap((k1-min_height)/max_height) for k1 in ran] )
        #print (ran, max_height, min_height)
        x = [x_orig[x1],]*20
        y = [y_orig[x1],]*20
        #print (len(Z_sgmesh))
        #ax.bar3d(x, y,Z_sgmesh, 0.0005, 0.0005, 0.1, color=rgba,zsort='average')
        
        ax.bar3d(x, y,Z_sgmesh, 0.0005, 0.0005, 0.1, color=rgba,zsort='average',norm=matplotlib.colors.LogNorm())
        
        #ax.set_zticks([])
        
    plt.savefig(outfile + 'pfig_%s.png'%k,dpi=100)
    #break
    plt.close()
    count +=1
    if count >4:
        break
"""

'\nx_orig = X_mesh\ny_orig = Y_mesh\n\ncentroid = []\nthick = [1,1,1]\nprint (cycles)\ncount = 0\nfor k in range(0,cycles,1):\n    \n    fig = plt.figure(figsize=(8,4))#subplots(1,1,)\n    ax = fig.gca(projection=\'3d\',adjustable=\'box\')\n    \n    y1 = y_orig\n    y2 = [i + 0.000001 for i in y1]\n    X = [x_orig,x_orig]\n    Y = [y1,y2]\n    \n    z = np.ones(len(x_orig))*1\n\n    Z = np.array([z,z])\n    data = np.linspace(1,0,len(x_orig))\n    Z_data = np.array([data,data])\n    \n    data_color = cm.jet(Z_data/np.amax(Z_data))\n    \n    #ran = [f*1 for f in flux_interpolate] #Discharge_stream\n    ran = Stream_C[Suffix[0]][k]\n    ran = [r*1e-3 if r>0 else 1e-12 for r in ran]\n    print (k, Suffix[0], len(ran))\n    sc = ax.scatter(X_mesh,Y_mesh,z,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())\n    \n    \n    plt.colorbar(sc, label=\'Concentration [kg/m$^3$]\',shrink=.96)#,aspect=10)\n   \n    #ax.set_zlim(0,0.0)\n    #ax.set_zticks([])\n 

In [25]:

#Write solute concentration images to a directory
#%matplotlib qt

x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]
print ('Cycles: ',cycles)
count = 0
for k in range(0,cycles,1):
    print ('Main', k)
    fig = plt.figure(figsize=(12,6))
    
    ax = fig.add_subplot(121)
    plt.tight_layout()
    
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*0
    
    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    
    #ran = [f*1 for f in flux_interpolate] #Discharge_stream
    ran = Stream_C[Suffix[0]][k]
    ran = [r*1e-3 if r>0 else 1e-12 for r in ran]
    #print (k, Suffix[0], len(ran))
    sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())
    
    plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
   
    #ax.set_zlim(0,0.0)
    #ax.set_zticks([])
    
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
    
    sub_c = []
    sub_x = []
    sub_y = []
    sub_top = []
    
    ax = fig.add_subplot(122)#, projection='3d')
    
    #ax.view_init(45, -15) #this looks okay
    
    for x1 in range(0,len(x_orig),2):
        #print (x1)
        sg_infile = os.path.join(path,'visdump_subgrid_%s_data.h5'%x1)
        
        ran = subgrid_C(sg_infile,sg_var='subgrid_%s-total_component_concentration'%x1,suff=Suffix[0])[k][::-1]
        ran = [r*1e-3 if r>0 else 1e-12 for r in ran]
        Z_sgmesh = np.arange(0.05,1.05,0.05)#,dtype='int')
        Z_sgmesh = np.linspace(0,1.,20)
        s2 = ran[-2]#np.mean(ran)
        sub_c.append(s2)
        sub_x.append(x_orig[x1])
        sub_y.append(y_orig[x1])
        
        width = depth = 0.001
        bottom, top = 0, s2
        
    cmap= cm.get_cmap('coolwarm') 
    min_height, max_height = 0, np.max(sub_c)+0.0001
    #print (sub_c)
    rgba = np.array([cmap((k1-min_height)/max_height) for k1 in sub_c] )
    #1
    #sc1 = ax.bar3d(sub_x, sub_y,bottom, width, depth, sub_c,color=rgba, zsort='average')#,norm=matplotlib.colors.LogNorm())
    #sc = ax.scatter(X_mesh,Y_mesh,z*0,marker=".")#,c=ran,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())
    #2
    sub_z = np.ones(len(sub_x))*1
    #print (sub_c)
    sc1 = ax.scatter(sub_x,sub_y,marker=".",c=sub_c,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())
    #sc = ax.scatter(X_mesh,Y_mesh,z,marker=".",c=sub_c,cmap=cm.coolwarm,vmin=1e-04,vmax=1,norm=matplotlib.colors.LogNorm())
    plt.colorbar(sc1, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
    
    """
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_zlim(0,.1)
    ax.set_zticks([])
    
    ax.w_zaxis.line.set_lw(0.)
    """
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
    
    plt.savefig(outfile + 'pfig_%s.png'%k,dpi=100)
    #break
    plt.close()
    count +=1
    
    if count >600:
        print ('Count: ',count)
        break

Cycles:  46
Main 0
Main 1
Main 2
Main 3
Main 4
Main 5
Main 6
Main 7
Main 8
Main 9
Main 10
Main 11
Main 12
Main 13
Main 14
Main 15
Main 16
Main 17
Main 18
Main 19
Main 20
Main 21
Main 22
Main 23
Main 24
Main 25
Main 26
Main 27
Main 28
Main 29
Main 30
Main 31
Main 32
Main 33
Main 34
Main 35
Main 36
Main 37
Main 38
Main 39
Main 40
Main 41
Main 42
Main 43
Main 44
Main 45


In [ ]:

from sparklines import sparklines
for line in sparklines([2, 7, 1,5,7,8,9]): 
    print (line)


In [ ]:
import re, os

def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

dir_files = os.listdir(path)

Files = sorted_nicely(dir_files)

Files = [f for f in Files if f.endswith('.png')]

#Files = Files[:60] + Files[450:520] #+ Files[37:70]
#Files = Files[:200] + Files[280:] 
img_array = []
for filename in Files[::2]:
    filename = path + filename
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    


In [ ]:
out = cv2.VideoWriter(path + 'stream-transport-coweeta.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/plots_1014/'
#%matplotlib tk
x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]

for k in range(cycles):
    fig, ax = plt.subplots(1,1,figsize=(8,4))
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*0

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    #ran = Discharge_stream
    ran = Concentration[140]#[k+120]
    ran = [r if r>0 else 1e-12 for r in ran]
    #print (ran)
    sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-3,vmax=1,norm=matplotlib.colors.LogNorm())
    #sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.tab10,vmin=0,vmax=1)# map=plt.cm.get_cmap('prism', 5))
    #plt.colorbar(sc, label='Discharge [cm$^3$/sec]',shrink=.96)#,aspect=10)
    plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
    
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
   
    #plt.savefig(path + 'figs_%s.png'%k,dpi=500)
    break
    plt.close()
    #break

In [ ]:
#Load all images
"""
import cv2
import numpy as np
import glob
 
img_array = []
for filename in glob.glob(path+'*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
"""



from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

from mpl_toolkits.mplot3d import Axes3D 

import numpy
import matplotlib 
import matplotlib.pyplot as plt


def plot_cylinder_element(x, y, z, dz, rx = 5, ry = 5, color = "b"):
    """
    x: left, right
    z: start height
    dz: height of cylinder
    rx, ry = radius of width (x) and depth (y)
    color = color

    Inspired by:
http://matplotlib.1069221.n5.nabble.com/plot-surface-shading-and-clipping-error-td14031.html
    """

    N = 100             # number of elements
    # a lower stride will give more faces. A cylinder with 4 faces is a cube :)
    # I think with N=100 and rstride=2, it will have 50 faces
    # cstride is the height, rstride the circle
    cstride_side = 1000 # only 1 element needed
    rstride_side = 1    # many elements to make a nice cylinder shape
    cstride_top = 10    
    rstride_top = 10

    # parameters of cylinder
    phi = numpy.linspace(0, 2 * numpy.pi, N) 
    _r = numpy.ones(N) 
    _h = numpy.linspace(0, 1, N) 

    # cylinder
    _x = rx * numpy.outer(numpy.cos(phi), _r) + x
    _y = ry * numpy.outer(numpy.sin(phi), _r) + y
    _z = dz * numpy.outer(numpy.ones(numpy.size(_r)), _h) + z
    ax.plot_surface(_x, _y, _z, rstride = rstride_side, cstride = cstride_side, linewidth = 0, alpha = 1, color = color) 

    # to cover the gaps between the faces, plot the cylinder again at a slightly smaller radius
    _x *= 0.99
    _y *= 0.99
    ax.plot_surface(_x, _y, _z, rstride = rstride_side + 1, cstride = cstride_side + 1, linewidth=0, alpha=1, color = color) 

    # top
    _x = rx * numpy.outer(numpy.cos(phi), _h) + x
    _y = ry * numpy.outer(numpy.sin(phi), _h) + y
    _z = numpy.zeros([N,N]) + z + dz + 0.1*0
    #print (_y)
    ax.plot_surface(_x, _y, _z,  rstride = rstride_top, cstride = cstride_top, linewidth = 0, alpha = 1, color = color) 

    # plot again with different stride to mask the gaps    
    ax.plot_surface(_x, _y, _z, rstride = rstride_side + 1, cstride = cstride_side + 1, linewidth=0, alpha=1, color = color) 


def plot_cylinder(x, y, z, rx = 5, ry = 0):
    """
    x: left-right for each cylinder
    z: list height difference (ie. not cumulative) 
    """
    # list with colors
    colors = ["b", "g", "r", "c", "y", "k"]
    # plot cylinder elements
    _z = 0
    #y = x
    for i in range(len(z)):
        plot_cylinder_element(x, y, _z, z[i], rx = rx, ry = ry, color = colors[i % len(colors)])  
        _z += z[i]


def cylinder_plot(x0, y0, z, r = 10, dr = 30):
    """
    z: list of different cylinders with for each a list height difference (ie. not cumulative)
    r: radius
    dr: distance between cylinders    
    """
    # different cylinders next to each other
    x = x0 #numpy.arange(len(z)) * dr
    # possible difference between width (x) and depth (y)
    rx = 1 #r
    ry = 1 #0*r
    y = y0
    # make cylinders
    for i in range(len(z)):
        print ('HERE1: ',i,x[i], z[i], rx,ry)
        plot_cylinder(x[i], y[i], z[i], rx = rx, ry = ry)


# close earlier plots
plt.close("all")

# make figure
fig = plt.figure() 
ax = Axes3D(fig) 

# set 3D-view
ax.view_init(elev = 1, azim = 280)

# make 3 cylinders, with a different number of elements
z1 = [0.1,]*10
x0 =[1,4,6]
y0 =[1,4,6]
print (z1)
#cylinder_plot(x0,y0, [[5, 10, 5], [3, 5], [1,2,3,4]])

cylinder_plot(x0,y0, [z1])

# set the labels
#ax.set_xlabel('X') 
#ax.set_ylabel('Y') 
#ax.set_zlabel('Z')

# show
#plt.show() 